In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import librosa
import librosa.display
import matplotlib.pyplot as plt
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# Parametry dla spektrogramów Mela
SAMPLE_RATE = 16000  # Częstotliwość próbkowania
N_MELS = 128  # Liczba filtrów Mela
N_FFT = 2048  # Długość okna FFT
HOP_LENGTH = 512  # Długość przeskoku między ramkami
DURATION = 3  # Maksymalna długość nagrania w sekundach

# Parametry dla modelu
BATCH_SIZE = 32
EPOCHS = 50
NUM_CLASSES = 6  # 6 emocji: złość, strach, szczęście, smutek, zaskoczenie, neutralny

def load_nemo_dataset():
    """
    Ładowanie datasetu nEMO z Hugging Face
    """
    print("Ładowanie datasetu nEMO...")
    dataset = load_dataset("amu-cai/nEMO")
    return dataset

def extract_melspectrogram(audio, sr=SAMPLE_RATE):
    """
    Ekstrakcja spektrogramu Mela z sygnału audio
    """
    # Przycinanie lub paddowanie do stałej długości
    if len(audio) > sr * DURATION:
        audio = audio[:sr * DURATION]
    else:
        audio = np.pad(audio, (0, max(0, sr * DURATION - len(audio))), 'constant')

    # Generowanie spektrogramu Mela
    mel_spectrogram = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_mels=N_MELS,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH
    )

    # Konwersja do decybeli i normalizacja 
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    mel_spectrogram = (mel_spectrogram - np.min(mel_spectrogram)) / (np.max(mel_spectrogram) - np.min(mel_spectrogram))
    return mel_spectrogram

def prepare_data(dataset):
    """
    Przygotowanie danych: ekstrakcja spektrogramów Mela i etykiet
    """
    print("Przygotowywanie danych...")

    # Mapowanie etykiet na indeksy
    emotion_mapping = {
        'anger': 0,        # złość
        'fear': 1,         # strach
        'happiness': 2,    # szczęście
        'sadness': 3,      # smutek
        'surprised': 4,     # zaskoczenie
        'neutral': 5       # neutralny
    }

    # Listy na dane
    mel_spectrograms = []
    labels = []

    # Przetwarzanie zestawu treningowego
    for sample in dataset['train']:
        audio = np.array(sample['audio']['array'])
        emotion = sample['emotion']

        if emotion in emotion_mapping:
            mel_spec = extract_melspectrogram(audio)
            mel_spectrograms.append(mel_spec)
            labels.append(emotion_mapping[emotion])

    # Konwersja do numpy arrays
    X = np.array(mel_spectrograms)
    y = np.array(labels)

    # Dostosowanie wymiarów dla VGG16 (wymaga 3 kanałów kolorów)
    # Replikacja spektrogramu Mela na 3 kanały (RGB)
    X = np.repeat(X[:, :, :, np.newaxis], 3, axis=3)

    # Podział na zbiory treningowe i testowe
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

    return X_train, X_val, X_test, y_train, y_val, y_test

def visualize_melspectrogram(mel_spectrogram, title):
    """
    Wizualizacja spektrogramu Mela
    """
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(
        mel_spectrogram[0, :, :, 0],  # Wybieramy pierwszy kanał z 3 kanałów RGB
        y_axis='mel',
        x_axis='time',
        sr=SAMPLE_RATE,
        hop_length=HOP_LENGTH
    )
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

def build_vgg16_model(input_shape, num_classes):
    """
    Budowanie modelu z wykorzystaniem pre-trenowanego VGG16
    """
    print("Budowanie modelu z wykorzystaniem pre-trenowanego VGG16...")

    # Ładowanie pre-trenowanego modelu VGG16 bez warstw w pełni połączonych
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Zamrożenie wag pre-trenowanego modelu
    for layer in base_model.layers:
        layer.trainable = False

    # Dodanie własnych warstw klasyfikacyjnych
    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Stworzenie finalnego modelu
    model = Model(inputs=base_model.input, outputs=predictions)

    # Kompilacja modelu
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def train_model(model, X_train, y_train, X_val, y_val):
    """
    Trenowanie modelu VGG16
    """
    print("Trenowanie modelu...")

    # Generowanie dodatkowych danych treningowych
    datagen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )

    # Callbacks
    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]

    # Trenowanie
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        steps_per_epoch=len(X_train) // BATCH_SIZE
    )

    return model, history

# Definicja emotion_names 
emotion_names = ['Złość', 'Strach', 'Szczęście', 'Smutek', 'Zaskoczenie', 'Neutralny']

def evaluate_model(model, X_test, y_test):
    """
    Ewaluacja modelu na zbiorze testowym z normalizowaną macierzą pomyłek
    """
    print("Ewaluacja modelu...")

    # Ewaluacja
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}")

    # Predykcje
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Macierz pomyłek
    from sklearn.metrics import confusion_matrix, classification_report
    cm = confusion_matrix(y_test, y_pred_classes)

    # Normalizacja macierzy pomyłek (wartości między 0 a 1)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Raport klasyfikacji
    print("Raport klasyfikacji:")
    print(classification_report(y_test, y_pred_classes, target_names=emotion_names))

    # Wizualizacja znormalizowanej macierzy pomyłek
    plt.figure(figsize=(10, 8))
    plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Znormalizowana macierz pomyłek')
    plt.colorbar()
    tick_marks = np.arange(len(emotion_names))
    plt.xticks(tick_marks, emotion_names, rotation=45)
    plt.yticks(tick_marks, emotion_names)

    # Dodanie wartości do macierzy pomyłek
    thresh = cm_normalized.max() / 2.
    for i in range(cm_normalized.shape[0]):
        for j in range(cm_normalized.shape[1]):
            plt.text(j, i, format(cm_normalized[i, j], '.2f'),  # Wyświetlamy do 2 miejsc po przecinku
                     horizontalalignment="center",
                     color="white" if cm_normalized[i, j] > thresh else "black")

    plt.ylabel('Rzeczywista etykieta')
    plt.xlabel('Przewidziana etykieta')
    plt.tight_layout()
    plt.show()

    return test_acc, test_loss, cm_normalized

def plot_training_history(history):
    """
    Wizualizacja historii treningu
    """
    plt.figure(figsize=(12, 4))

    # Wykres dokładności
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Dokładność treningowa')
    plt.plot(history.history['val_accuracy'], label='Dokładność walidacyjna')
    plt.title('Dokładność modelu')
    plt.ylabel('Dokładność')
    plt.xlabel('Epoka')
    plt.legend(loc='lower right')

    # Wykres funkcji kosztu
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Koszt treningowy')
    plt.plot(history.history['val_loss'], label='Koszt walidacyjny')
    plt.title('Funkcja kosztu modelu')
    plt.ylabel('Koszt')
    plt.xlabel('Epoka')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

def save_model(model, history, filename='emotion_detection_model.h5'):
    """
    Zapisywanie modelu i historii
    """
    print(f"Zapisywanie modelu do pliku {filename}...")
    model.save(filename)

    # Zapisywanie historii
    history_dict = {
        'accuracy': history.history['accuracy'],
        'val_accuracy': history.history['val_accuracy'],
        'loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    }

    np.save('training_history.npy', history_dict)
    print("Model i historia zapisane pomyślnie.")

def predict_emotion(model, audio_path):
    """
    Predykcja emocji dla nowego pliku audio
    """
    
    # Wczytanie audio
    audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE)

    # Ekstrakcja spektrogramu Mela
    mel_spec = extract_melspectrogram(audio)

    # Przygotowanie do predykcji - replikacja na 3 kanały dla VGG16
    mel_spec = np.repeat(mel_spec[:, :, np.newaxis], 3, axis=2)
    mel_spec = mel_spec.reshape(1, N_MELS, mel_spec.shape[1], 3)

    # Predykcja
    prediction = model.predict(mel_spec)
    predicted_class = np.argmax(prediction[0])

    # Wyniki
    print(f"Przewidziana emocja: {emotion_names[predicted_class]}")
    print("Prawdopodobieństwa:")
    for i, emotion in enumerate(emotion_names):
        print(f"{emotion}: {prediction[0][i]:.4f}")

    return predicted_class, prediction[0]

# Faktyczne wykonanie programu - uruchomienie całego procesu
if __name__ == "__main__":
    # Ładowanie datasetu
    dataset = load_nemo_dataset()

    # Przygotowanie danych
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(dataset)

    # Sprawdzenie kształtu danych
    print(f"Kształt danych treningowych: {X_train.shape}")
    print(f"Kształt danych walidacyjnych: {X_val.shape}")
    print(f"Kształt danych testowych: {X_test.shape}")

    # Wizualizacja przykładowego spektrogramu Mela
    visualize_melspectrogram(X_train[:1], "Przykładowy spektrogram Mela")

    # Budowa modelu
    input_shape = (N_MELS, X_train.shape[2], 3)  # 3 kanały dla VGG16
    model = build_vgg16_model(input_shape, NUM_CLASSES)
    model.summary()

    # Trenowanie modelu
    model, history = train_model(model, X_train, y_train, X_val, y_val)

    # Ewaluacja modelu
    test_acc, test_loss, confusion_matrix = evaluate_model(model, X_test, y_test)

    # Wizualizacja wyników
    plot_training_history(history)

    # Zapisanie modelu
    save_model(model, history)

    print("Proces zakończony!")